# Data Preprocessing

In [88]:
from scipy.io import arff
from io import StringIO
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from imblearn.over_sampling import SMOTE
import pandas as pd

In [89]:
#This loads the file containing the data from the UCI Machine Learning repository
data_file = '../data/ThoraricSurgery.arff'
data, meta = arff.loadarff(data_file)
meta

Dataset: 'Thoracic_Surgery_Data'
	DGN's type is nominal, range is ('DGN3', 'DGN2', 'DGN4', 'DGN6', 'DGN5', 'DGN8', 'DGN1')
	PRE4's type is numeric
	PRE5's type is numeric
	PRE6's type is nominal, range is ('PRZ2', 'PRZ1', 'PRZ0')
	PRE7's type is nominal, range is ('T', 'F')
	PRE8's type is nominal, range is ('T', 'F')
	PRE9's type is nominal, range is ('T', 'F')
	PRE10's type is nominal, range is ('T', 'F')
	PRE11's type is nominal, range is ('T', 'F')
	PRE14's type is nominal, range is ('OC11', 'OC14', 'OC12', 'OC13')
	PRE17's type is nominal, range is ('T', 'F')
	PRE19's type is nominal, range is ('T', 'F')
	PRE25's type is nominal, range is ('T', 'F')
	PRE30's type is nominal, range is ('T', 'F')
	PRE32's type is nominal, range is ('T', 'F')
	AGE's type is numeric
	Risk1Yr's type is nominal, range is ('T', 'F')

In [90]:
outcomes_df = pd.DataFrame.from_records(data)
nominal_features = ['DGN', 'PRE6',  'PRE14']
TF_features = ['PRE7', 'PRE8', 'PRE9', 'PRE10', 'PRE11', 'PRE17', 'PRE19', 'PRE19',
              'PRE25', 'PRE30', 'PRE32', 'Risk1Yr']
numeric_features = ['PRE4', 'PRE5', 'AGE']
nominal_dict = {feature: [] for feature in nominal_features}
TF_dict = {feature: [] for feature in TF_features}

In [91]:
# This collects all of the categorical features and true/false features so that they can be preprocessed
for feature in nominal_features:
    nominal_dict[feature][:] = (pd.Series(outcomes_df[feature].values).unique().tolist())
for feature in TF_features:
    TF_dict[feature][:] = (pd.Series(outcomes_df[feature].values).unique().tolist())

## One Hot Encoding

In [92]:
outcomes_df.head(5)

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,b'DGN2',2.88,2.16,b'PRZ1',b'F',b'F',b'F',b'T',b'T',b'OC14',b'F',b'F',b'F',b'T',b'F',60.0,b'F'
1,b'DGN3',3.40,1.88,b'PRZ0',b'F',b'F',b'F',b'F',b'F',b'OC12',b'F',b'F',b'F',b'T',b'F',51.0,b'F'
2,b'DGN3',2.76,2.08,b'PRZ1',b'F',b'F',b'F',b'T',b'F',b'OC11',b'F',b'F',b'F',b'T',b'F',59.0,b'F'
3,b'DGN3',3.68,3.04,b'PRZ0',b'F',b'F',b'F',b'F',b'F',b'OC11',b'F',b'F',b'F',b'F',b'F',54.0,b'F'
4,b'DGN3',2.44,0.96,b'PRZ2',b'F',b'T',b'F',b'T',b'T',b'OC11',b'F',b'F',b'F',b'T',b'F',73.0,b'T'


In [93]:
outcomes_df
#This for loop performs two of the preprocessing steps in our pipeline which are: one hot encoding
# and conversion of true/false strings to 0 and 1 respectively
for feature in outcomes_df.columns.values:
    new_column = []
    # This loop converts the categorical features into one hot encodings
    for i, row in outcomes_df.iterrows():
        if feature in nominal_features:
            encoding = np.zeros(len(nominal_dict[feature])).tolist()
            index = nominal_dict[feature].index(row[feature])
            encoding[index] = 1
            new_column.append(str(encoding))
        # If the feature is a true/false feature then it is converted to a binary value
        if feature in TF_features:
            outcomes_df.loc[i,feature] = TF_dict[feature].index(row[feature])
    if feature in nominal_features:
        outcomes_df.drop([feature], axis=1)
        outcomes_df[feature] = pd.DataFrame(new_column, columns=[feature])

In [94]:
outcomes_df

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2.88,2.16,"[1, 0.0, 0.0]",0,0,0,0,0,"[1, 0.0, 0.0, 0.0]",0,0,0,0,0,60.0,0
1,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.40,1.88,"[0.0, 1, 0.0]",0,0,0,1,1,"[0.0, 1, 0.0, 0.0]",0,0,0,0,0,51.0,0
2,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.76,2.08,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,59.0,0
3,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.68,3.04,"[0.0, 1, 0.0]",0,0,0,1,1,"[0.0, 0.0, 1, 0.0]",0,0,0,1,0,54.0,0
4,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.44,0.96,"[0.0, 0.0, 1]",0,1,0,0,0,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,73.0,1
5,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.48,1.88,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,1,0,51.0,0
6,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",4.36,3.28,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 1, 0.0, 0.0]",1,0,0,0,0,59.0,1
7,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",3.19,2.50,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,1,0,0,66.0,1
8,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.16,2.64,"[0.0, 0.0, 1]",0,0,0,0,0,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,68.0,0
9,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.32,2.16,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,54.0,0


In [105]:
df = outcomes_df.groupby(['Risk1Yr'], sort=False).size().reset_index(name='Count')

In [106]:
df

,Risk1Yr,Count
0,0,400
1,1,350


In [97]:
# These functions sepsrate the dataset into the two classes so that the imbalance within the dataset
# can be observed and used for the following upsampling
Risk1Yr_1_df = outcomes_df.loc[outcomes_df['Risk1Yr'] == 1]
Risk1Yr_0_df = outcomes_df.loc[outcomes_df['Risk1Yr'] == 0]
Risk1Yr_0_df

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2.88,2.16,"[1, 0.0, 0.0]",0,0,0,0,0,"[1, 0.0, 0.0, 0.0]",0,0,0,0,0,60.0,0
1,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.40,1.88,"[0.0, 1, 0.0]",0,0,0,1,1,"[0.0, 1, 0.0, 0.0]",0,0,0,0,0,51.0,0
2,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.76,2.08,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,59.0,0
3,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.68,3.04,"[0.0, 1, 0.0]",0,0,0,1,1,"[0.0, 0.0, 1, 0.0]",0,0,0,1,0,54.0,0
5,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.48,1.88,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,1,0,51.0,0
8,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.16,2.64,"[0.0, 0.0, 1]",0,0,0,0,0,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,68.0,0
9,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.32,2.16,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,54.0,0
10,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.56,2.32,"[0.0, 1, 0.0]",0,1,0,0,1,"[0.0, 1, 0.0, 0.0]",0,0,0,1,0,60.0,0
11,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",4.28,4.44,"[1, 0.0, 0.0]",0,0,0,1,1,"[0.0, 1, 0.0, 0.0]",0,0,0,0,0,58.0,0
12,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.00,2.36,"[1, 0.0, 0.0]",0,0,0,0,0,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,68.0,0


In [98]:
# This for loop upsamples the patients who died within 1 year of their procedures since
# the dataset had a class imbalance and results in an approximate 1:1 ratio of the dataset
for i in range(4):
    outcomes_df = outcomes_df.append(Risk1Yr_1_df)
outcomes_df

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
0,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",2.88,2.16,"[1, 0.0, 0.0]",0,0,0,0,0,"[1, 0.0, 0.0, 0.0]",0,0,0,0,0,60.0,0
1,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.40,1.88,"[0.0, 1, 0.0]",0,0,0,1,1,"[0.0, 1, 0.0, 0.0]",0,0,0,0,0,51.0,0
2,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.76,2.08,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,59.0,0
3,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.68,3.04,"[0.0, 1, 0.0]",0,0,0,1,1,"[0.0, 0.0, 1, 0.0]",0,0,0,1,0,54.0,0
4,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.44,0.96,"[0.0, 0.0, 1]",0,1,0,0,0,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,73.0,1
5,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.48,1.88,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,1,0,51.0,0
6,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",4.36,3.28,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 1, 0.0, 0.0]",1,0,0,0,0,59.0,1
7,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",3.19,2.50,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,1,0,0,66.0,1
8,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",3.16,2.64,"[0.0, 0.0, 1]",0,0,0,0,0,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,68.0,0
9,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.32,2.16,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,54.0,0


In [99]:
# outcomes_df = Risk1Yr_0_df.append(Risk1Yr_1_df )
# outcomes_df.shape

## Train Test Split

In [100]:
# Using Scikit-learn we split the dataset into an 80-20 training and test split that uses stratification to ensure
# that the proportion of classes in the dataset is maintained in both the training and test sets
outcomes_train, outcomes_test = train_test_split(outcomes_df, test_size=0.2, stratify=outcomes_df['Risk1Yr'].values.tolist())
outcomes_train_df = pd.DataFrame(outcomes_train)
outcomes_test_df = pd.DataFrame(outcomes_test)

In [101]:
# This function is used for standardizing the features to reduce the variance and improve the neural network
# performance
def standardization(x, mean, std):
    z_scores_np = (x - mean) / std
    return z_scores_np

# This function rescales the features using the standardization function for the training set
def train_standard_loop(keys):
    for key in keys:
        x_np = outcomes_train_df[key].values
        x_np_mean = x_np.mean()
        x_np_std = x_np.std()
        outcomes_train_df[key] = outcomes_train_df[key].apply(standardization,args=(x_np_mean,x_np_std))

# This function rescales the features using the standardization function for the test set
def test_standard_loop(keys):
    for key in keys[1:10]:
        x_np = outcomes_test_df[key].values
        x_np_mean = x_np.mean()
        x_np_std = x_np.std()
        outcomes_test_df[key] = outcomes_test_df[key].apply(standardization,args=(x_np_mean,x_np_std))

train_standard_loop(list(['PRE4', 'PRE5', 'AGE']))
test_standard_loop(list(['PRE4', 'PRE5', 'AGE']))

In [102]:
outcomes_train_df

,DGN,PRE4,PRE5,PRE6,PRE7,PRE8,PRE9,PRE10,PRE11,PRE14,PRE17,PRE19,PRE25,PRE30,PRE32,AGE,Risk1Yr
158,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.200227,-0.162578,"[1, 0.0, 0.0]",0,1,0,0,0,"[0.0, 1, 0.0, 0.0]",0,0,1,0,0,-0.229806,0
169,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.983352,-0.198304,"[1, 0.0, 0.0]",0,1,1,0,1,"[0.0, 0.0, 0.0, 1]",0,0,0,1,0,-0.341633,1
465,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0.778679,-0.182426,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 0.0, 1]",0,0,0,0,0,-0.006151,0
51,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",0.925515,-0.107003,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,0.664815,0
296,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.934406,-0.210213,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 1, 0.0, 0.0]",0,0,0,0,0,-0.900772,1
191,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.689680,-0.154638,"[0.0, 1, 0.0]",0,0,0,1,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,-0.341633,1
40,"[0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0.680788,-0.097079,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 0.0, 1, 0.0]",0,0,0,0,0,-0.341633,1
134,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",1.953365,-0.047458,"[1, 0.0, 0.0]",0,0,0,0,1,"[0.0, 1, 0.0, 0.0]",0,0,0,0,0,-0.788944,0
404,"[1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",-0.347063,-0.170517,"[0.0, 1, 0.0]",0,0,0,0,1,"[0.0, 1, 0.0, 0.0]",0,0,0,0,0,-0.677116,1
304,"[0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0]",2.344928,0.031934,"[0.0, 1, 0.0]",0,0,0,1,1,"[0.0, 1, 0.0, 0.0]",0,0,0,0,0,-0.788944,0


In [103]:
outcomes_train_df.to_csv(path_or_buf='outcomes_data_train.csv',index=False)
outcomes_test_df.to_csv(path_or_buf='outcomes_data_test.csv',index=False)